In [1]:
from helpers import load_all_datasets, compose, str_rep_vdb, str_rep_no_steps, CBRDatabase, str_rep_vdb_ing, str_rep_vdb_nam 
import pandas as pd

additional_cbr_1, additional_cbr_2, additional_validation, cbr_augmentation, cbr_database, model_training, validation = load_all_datasets()

cbr_database = cbr_database.set_index("id")

EX_APP = "nf"

INITIAL_C = 0
GENERATED_C = 1

DATABASE_NAME = f"cbr_database_{INITIAL_C}_{GENERATED_C}_{EX_APP}"
OUTPUT_FILE = f"cbr_{INITIAL_C}_{GENERATED_C}_{EX_APP}_gpt2_input"

IX_FILE = "cbr_augmentation_12_gpt2_results_p_ix.csv"
dfix = pd.read_csv(f"{IX_FILE}")


In [2]:
# cbr_db = pd.concat([cbr_database, additional_cbr_1])
cbr_database = cbr_database[:int(INITIAL_C*1000)]
dfix = dfix[:int(GENERATED_C*1000)]
dfix = dfix.set_index("id")
cbr_db_ix = pd.concat([cbr_database, dfix])
# cbr_db_ix = cbr_db_ix.set_index("id")

cbr_reps = str_rep_vdb(cbr_db_ix)
len(cbr_reps)

1000

In [3]:
meta = CBRDatabase.create_metadatas([int(e) for e in list(cbr_db_ix.index.values)], {"dataset":DATABASE_NAME})
vdb = CBRDatabase(cbr_reps, metadatas=meta, db_directory=f"dataset_vector/{DATABASE_NAME}", initialize_new=True)

# vdb = CBRDatabase(db_directory=f"dataset_vector/{DATABASE_NAME}", initialize_new=False)

print(len(vdb.get_texts()))
print(vdb.similarity_search(query="test", k=2))

bert_load_from_file: gguf version     = 2
bert_load_from_file: gguf alignment   = 32
bert_load_from_file: gguf data offset = 695552
bert_load_from_file: model name           = BERT
bert_load_from_file: model architecture   = bert
bert_load_from_file: model file type      = 1
bert_load_from_file: bert tokenizer vocab = 30522
1000
[Document(page_content='name: easy no fail chocolate fudge no thermometer ; ingredients: sugar, evaporated milk, butter, salt, mini marshmallows, semi-sweet chocolate chips, walnuts, vanilla ; preparation:', metadata={'dataset': 'cbr_database_0_1_nf', 'recipe_id': 80447}), Document(page_content='name: blt club sandwich ; ingredients: bread, mayonnaise, american cheese, tomatoes, lettuce, crisp bacon ; preparation:', metadata={'dataset': 'cbr_database_0_1_nf', 'recipe_id': 267197})]


In [4]:
tem = vdb.similarity_search(query="akjsjd", k=5)
recipe_ixs = [e.metadata[CBRDatabase.RECIPE_ID_KEY] for e in tem]
recipe_ixs

[168244, 385, 38545, 132112, 307186]

In [5]:
# generowanie inputów - 850 tokenów 
from pprint import pprint
import random
from transformers import LlamaTokenizerFast

tokenizer = LlamaTokenizerFast.from_pretrained("hf-internal-testing/llama-tokenizer")

inputs = []
pe_recipes = []
target_recipe = []
lens = []

for i, v_recipe in validation.iterrows():
    while True:
        tem = vdb.similarity_search(query=str_rep_no_steps(v_recipe), k=5)
        recipe_ixs = [e.metadata[CBRDatabase.RECIPE_ID_KEY] for e in tem]
        
        #dev
        # print(tem)
        # composed = compose(cbr_db_ix.loc[recipe_ixs[0:2]], v_recipe)
        # print(recipe_ixs)
        # print(composed)
        # pprint(composed.split("name: "))
        # raise
        #dev

        gen_input = compose(cbr_db_ix.loc[recipe_ixs], v_recipe)
        br = False
        if len(tokenizer.encode(gen_input)) <= 850: 
            br = True
        else:
            for ii in list(range(4, -1, -1)):
                gen_input = compose(cbr_db_ix.loc[recipe_ixs[:ii] + recipe_ixs[ii+1:]], v_recipe)
                if len(tokenizer.encode(gen_input)) <= 850: 
                    br = True
                    print(f"1 - {len(tokenizer.encode(gen_input))}")
                    break
            if br: break
            for ii in list(range(3, -1, -1)):
                gen_input = compose(cbr_db_ix.loc[recipe_ixs[:ii] + recipe_ixs[ii+2:]], v_recipe)
                if len(tokenizer.encode(gen_input)) <= 850: 
                    br = True
                    print(f"2 - {len(tokenizer.encode(gen_input))}")
                    break
            if br: break
            for ii in list(range(2, -1, -1)):
                gen_input = compose(cbr_db_ix.loc[recipe_ixs[:ii] + recipe_ixs[ii+3:]], v_recipe)
                if len(tokenizer.encode(gen_input)) <= 850: 
                    br = True
                    print(f"2 - {len(tokenizer.encode(gen_input))}")
                    break
        
        if not br:
            print(cbr_db_ix.loc[recipe_ixs].steps.values)
            print(f"REPEAT - {len(tokenizer.encode(gen_input))}")
        else:
            lens.append(len(tokenizer.encode(gen_input)))
            break
            
        
    pe_recipes.append(recipe_ixs)
    target_recipe.append(v_recipe["id"])
    inputs.append(gen_input)

/home/maciej/anaconda3/envs/ml_lm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[Document(page_content='name: white cream bars ; ingredients: flour, sugar, butter, nuts, cream cheese, powdered sugar, milk, cool whip, heath candy bars ; preparation:', metadata={'dataset': 'cbr_database_0_1_nf', 'recipe_id': 255645}), Document(page_content='name: rum mocha chocolate sauce ; ingredients: butter, granulated sugar, unsweetened cocoa powder, dark rum, heavy cream, salt, dry instant coffee, vanilla extract ; preparation:', metadata={'dataset': 'cbr_database_0_1_nf', 'recipe_id': 108617}), Document(page_content='name: plantation bars ; ingredients: sugar, corn syrup, chunky peanut butter, cheerios toasted oat cereal, rice krispies ; preparation:', metadata={'dataset': 'cbr_database_0_1_nf', 'recipe_id': 272639}), Document(page_content='name: fat free chocolate banana muffins ; ingredients: whole wheat flour, sugar, unsweetened cocoa, baking powder, baking soda, salt, bananas, nonfat milk, vinegar ; preparation:', metadata={'dataset': 'cbr_database_0_1_nf', 'recipe_id': 20

RuntimeError: No active exception to reraise

In [ ]:
df = pd.DataFrame()
df["input"] = inputs
df["pe_recipes"] = pe_recipes
df["target_recipe"] = target_recipe
df.to_csv(f"input_datasets/{OUTPUT_FILE}.csv", index=False)

In [ ]:
vdb.similarity_search(query="str_rep_no_steps(v_recipe)", k=4)

[Document(page_content='name: stroganoff 1953 ; ingredients: round steaks, flour, salt, pepper, butter, fresh mushrooms, onion, consomme, vinegar, sour cream ; preparation:', metadata={'dataset': 'cbr_database_1_1_nf', 'recipe_id': 464216}),
 Document(page_content='name: homemade tvp crumbles ; ingredients: textured vegetable protein, wheat germ, curry powder, cayenne pepper, sea salt, braggs liquid aminos, boiling water, whole wheat flour, olive oil, garlic ; preparation:', metadata={'dataset': 'cbr_database_1_1_nf', 'recipe_id': 513}),
 Document(page_content='name: make ahead mashed potatoes for crock pot ; ingredients: potatoes, sour cream, cream cheese, butter, chives, salt and pepper, paprika ; preparation:', metadata={'dataset': 'cbr_database_1_1_nf', 'recipe_id': 71205}),
 Document(page_content='name: crock pot savory pork roast ; ingredients: pork loin roast, onion, bay leaf, soy sauce, garlic powder ; preparation:', metadata={'dataset': 'cbr_database_1_1_nf', 'recipe_id': 2038

In [ ]:
from collections import Counter


Counter(df.input.apply(lambda x: x.split("name:")).apply(lambda x: len(x)))

Counter({6: 1095, 7: 1009, 5: 19, 4: 2})